### Fraud detection with creditcard 

#### Big data processing with PySpark

The dataset for this exercise contains transactions made in September 2013 by European bank cardholders and includes 492 fraudulent transactions among 284,807 transactions. The dataset is very unbalanced, with positive (fraudulent) classes representing 0.172% of all transactions. The explanatory variables are anonymized and represented by the first thirty columns. The target variable is stored in the last column.

    The purpose of this exercise is to implement and train a random forest model with PySpark to detect fraudulent       transactions.

Principle of random forests Random forests are composed of a set of decision trees. These trees are distinguished from each other by the subsample of data on which they are trained on. These subsamples are randomly drawn from the initial data set.

The principle of operation of random forests is simple: many small classification trees are produced on a random fraction of the data. Random Forest votes classification trees in order to deduce the order and importance of the explanatory variables.

First, we'll build a Spark session named spark


In [1]:
# Import de SparkSession et SparkContext
from pyspark.sql import SparkSession

# Construction d'une Session Spark
spark = SparkSession \
    .builder \
    .appName("project pyspark") \
    .getOrCreate()
    
spark

Import creditcard.csv  file as DataFrame called df_raw
show an extract of DataFrame df_raw  

In [2]:
# Loading file 'creditcard.csv.csv'
df_raw = spark.read.csv(r'C:\Users\ShahulData\Documents\MiniProjects\PySpark\creditcard.csv', header = True)

# sample view of the data frame
df_raw.sample(False, 0.001 , seed = 222).toPandas()


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,1673,-0.840507697843278,-0.127979478244112,-0.432871262021807,0.0382653360716287,1.43346912531201,4.20419095124925,1.19155961893992,0.883840227213036,-0.730660416322823,...,-0.0934678400181341,-0.966072989147504,0.832084697064644,0.897323013373984,0.129324765815733,-0.593823834843939,0.196177724160185,0.128839216454571,359.89,0
1,2524,1.27413951703819,0.0399472682397259,-0.853533093097489,0.0561301601574641,2.00234310416062,3.52697529720514,-0.656805452727263,0.887438855810473,-0.00253797321628542,...,-0.133406738979587,-0.510320468239247,0.000638187234731824,0.974259987212245,0.574142977488771,-0.463084584112491,0.0371935139266541,0.0243029488233164,1,0
2,4496,-0.943127705823875,1.35547185227778,1.42496234175629,1.55044085884284,0.0629008885286155,0.100756105492152,0.203064943493641,0.346003614706086,0.613169533283304,...,-0.102276361256562,0.0573096155985353,-0.126256205351794,0.0569787456598689,0.143996417527458,-0.24020338642191,-0.122503014935466,0.0322568207894074,10.24,0
3,4756,1.01065845785938,-0.0873035633529374,-0.0072801292197235,1.45634157835067,0.296802124502142,0.484381619470783,0.106762876254527,-0.0586494085285677,1.63935026619275,...,-0.247629300520682,-0.494889161836857,-0.27801427659292,-0.969975388085556,0.729247497976511,-0.301377232506878,-0.0108404158892766,0.0198507343544592,126.43,0
4,6745,-0.413972310221646,1.0422110631202,1.36868585863346,0.242157218543727,0.163812669072657,-0.506044988492596,0.452030783403474,0.0664916258712286,1.10355021062782,...,-0.396411419036506,-0.881245318728687,0.0672464426954568,-0.0787302680738991,-0.300997047345715,0.0675628507416014,0.222296569402213,0.0864793701773414,10.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,167552,-0.457361832105007,1.51330353419297,-1.5012475172907,-1.09189093405213,0.953124671592645,-1.24191384190482,1.17565429518484,0.0209603529232136,-0.291500621266931,...,0.189351453955173,0.61345573400365,-0.183804586172698,0.65357739534506,0.381477066211741,0.155285704495636,0.0443976174908527,0.0369721776344707,3.7,0
278,168626,-0.531484031968166,1.45403721943648,0.405257293735003,3.97623813272114,1.30702112413709,2.93424588074856,-0.278865257485146,1.1862257325857,-2.17237519118165,...,0.341547246661056,1.10257633692446,-0.0118317367758385,-0.931691063223921,-0.791515635776013,0.50255352633416,0.399236484650498,0.176081403953609,20.8,0
279,169133,1.51180612289501,-1.56859374565907,-1.7569720424767,-0.233570836769656,-0.572955812414352,-0.671880200156777,0.24183806446196,-0.390942564566665,-0.862751629786906,...,0.128485662569414,0.247123365426789,-0.278349845613205,-0.301335491106414,0.0750929610132078,-0.145569240194975,-0.0561594191297922,-0.0065570275720651,340.13,0
280,170266,1.98062267437771,-0.164161111744648,-0.830005317487824,0.443089344959146,-0.460019755246761,-1.23883948538416,0.0164427373106953,-0.160035489236051,0.413225228886968,...,-0.213947943719839,-0.574217416482922,0.411735620816777,0.518002011698372,-0.441375054977864,0.14736781426513,-0.0809771282669993,-0.0669646228348851,0.89,0


Create a df DataFrame from df_raw by changing the target variable columns to double and the target variable class to int
Display sample variable schema of df

    A good practice is to put the variable to predict or the target variable in the first column

In [3]:
cols = list(df_raw.columns)
cols = [cols[-1]]  + cols[0:30]
df_raw = df_raw[cols]
df_raw.sample(False, 0.001 , seed = 222).toPandas()

,Class,Time,V1,V2,V3,V4,V5,V6,V7,V8,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,0,1673,-0.840507697843278,-0.127979478244112,-0.432871262021807,0.0382653360716287,1.43346912531201,4.20419095124925,1.19155961893992,0.883840227213036,...,1.05182380528198,-0.0934678400181341,-0.966072989147504,0.832084697064644,0.897323013373984,0.129324765815733,-0.593823834843939,0.196177724160185,0.128839216454571,359.89
1,0,2524,1.27413951703819,0.0399472682397259,-0.853533093097489,0.0561301601574641,2.00234310416062,3.52697529720514,-0.656805452727263,0.887438855810473,...,-0.0162199506926031,-0.133406738979587,-0.510320468239247,0.000638187234731824,0.974259987212245,0.574142977488771,-0.463084584112491,0.0371935139266541,0.0243029488233164,1
2,0,4496,-0.943127705823875,1.35547185227778,1.42496234175629,1.55044085884284,0.0629008885286155,0.100756105492152,0.203064943493641,0.346003614706086,...,-0.132191974770196,-0.102276361256562,0.0573096155985353,-0.126256205351794,0.0569787456598689,0.143996417527458,-0.24020338642191,-0.122503014935466,0.0322568207894074,10.24
3,0,4756,1.01065845785938,-0.0873035633529374,-0.0072801292197235,1.45634157835067,0.296802124502142,0.484381619470783,0.106762876254527,-0.0586494085285677,...,0.0359354351469093,-0.247629300520682,-0.494889161836857,-0.27801427659292,-0.969975388085556,0.729247497976511,-0.301377232506878,-0.0108404158892766,0.0198507343544592,126.43
4,0,6745,-0.413972310221646,1.0422110631202,1.36868585863346,0.242157218543727,0.163812669072657,-0.506044988492596,0.452030783403474,0.0664916258712286,...,-0.0295724753933139,-0.396411419036506,-0.881245318728687,0.0672464426954568,-0.0787302680738991,-0.300997047345715,0.0675628507416014,0.222296569402213,0.0864793701773414,10.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,0,167552,-0.457361832105007,1.51330353419297,-1.5012475172907,-1.09189093405213,0.953124671592645,-1.24191384190482,1.17565429518484,0.0209603529232136,...,-0.0276714180195565,0.189351453955173,0.61345573400365,-0.183804586172698,0.65357739534506,0.381477066211741,0.155285704495636,0.0443976174908527,0.0369721776344707,3.7
278,0,168626,-0.531484031968166,1.45403721943648,0.405257293735003,3.97623813272114,1.30702112413709,2.93424588074856,-0.278865257485146,1.1862257325857,...,0.217455384012756,0.341547246661056,1.10257633692446,-0.0118317367758385,-0.931691063223921,-0.791515635776013,0.50255352633416,0.399236484650498,0.176081403953609,20.8
279,0,169133,1.51180612289501,-1.56859374565907,-1.7569720424767,-0.233570836769656,-0.572955812414352,-0.671880200156777,0.24183806446196,-0.390942564566665,...,0.113332637755554,0.128485662569414,0.247123365426789,-0.278349845613205,-0.301335491106414,0.0750929610132078,-0.145569240194975,-0.0561594191297922,-0.0065570275720651,340.13
280,0,170266,1.98062267437771,-0.164161111744648,-0.830005317487824,0.443089344959146,-0.460019755246761,-1.23883948538416,0.0164427373106953,-0.160035489236051,...,-0.305339780941641,-0.213947943719839,-0.574217416482922,0.411735620816777,0.518002011698372,-0.441375054977864,0.14736781426513,-0.0809771282669993,-0.0669646228348851,0.89


In [4]:
# we inferred correct types of column
from pyspark.sql.functions import col
exprs = [col(c).cast("double") for c in df_raw.columns[1:31]]
df = df_raw.select(df_raw.Class.cast('int'), *exprs)

#df.columns
#df.dtypes

Delete the rows containing missing values from the DataFrame df

Create an rdd rdd_ml separating the variable to explain from the features (to be put in DenseVector form)

Create a DataFrame df_ml containing our database under two variables : 'labels' and 'features

In [5]:
### Get count of nan or missing values in pyspark
from pyspark.sql.functions import isnan, when, count, col
df.select([count(when(isnan(c), c)).alias(c) for c in df.columns]).show()
'''there is no null values other wise we can use df.na.drop()'''


+-----+----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+------+
|Class|Time| V1| V2| V3| V4| V5| V6| V7| V8| V9|V10|V11|V12|V13|V14|V15|V16|V17|V18|V19|V20|V21|V22|V23|V24|V25|V26|V27|V28|Amount|
+-----+----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+------+
|    0|   0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|     0|
+-----+----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+------+



'there is no null values other wise we can use df.na.drop()'

In [6]:
# Import DenseVector from package pyspark.ml.linalg
from pyspark.ml.linalg import DenseVector

''' our df is fully numeric which facilitates a non-indexing exclusion step'''

# Creation of a DenseVector variable containing the features through the RDD structure
rdd_ml = df.rdd.map(lambda x: (x[0], DenseVector(x[1:])))

# transformation into DataFrame and namind variables to obtain a base in svmlib form
df_ml = spark.createDataFrame(rdd_ml, ['label', 'features'])

# show an extract
df_ml.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|    0|[0.0,-1.359807133...|
|    0|[0.0,1.1918571113...|
|    0|[1.0,-1.358354061...|
|    0|[1.0,-0.966271711...|
|    0|[2.0,-1.158233093...|
|    0|[2.0,-0.425965884...|
|    0|[4.0,1.2296576345...|
|    0|[7.0,-0.644269442...|
|    0|[7.0,-0.894286082...|
|    0|[9.0,-0.338261752...|
|    0|[10.0,1.449043781...|
|    0|[10.0,0.384978215...|
|    0|[10.0,1.249998742...|
|    0|[11.0,1.069373587...|
|    0|[12.0,-2.79185476...|
|    0|[12.0,-0.75241704...|
|    0|[12.0,1.103215435...|
|    0|[13.0,-0.43690507...|
|    0|[14.0,-5.40125766...|
|    0|[15.0,1.492935976...|
+-----+--------------------+
only showing top 20 rows



Create two DataFrames called train and test each containing respectively 80% and 20% of the data

Create a Random Forest classifier called clf

Learn the random forest model on the training set

Learning massive data typically takes a long run time. The cell below should run in three minutes in the worst case.

In [8]:
# Decomposition of the data into two sets of training and test
# by default the sample is randomly distributed
train, test = df_ml.randomSplit([0.8, 0.2], seed=222)

# Creation of a classifier
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(labelCol="indexedLabel",
                            featuresCol="indexedFeatures",
                            predictionCol='prediction', seed = 222)


In [9]:
from time import time
t0 = time()

# Creation of tranformators
from pyspark.ml.feature import VectorIndexer, StringIndexer, IndexToString

labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(df_ml)
featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures").fit(df_ml)

# Creation of a transformer to restore the labels of the predictions
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)
tt = time() - t0
print("Réalisé en {} secondes".format(round(tt,3)))

Réalisé en 18.443 secondes


In [ ]:
from time import time
t0 = time()

# Creation of a  Pipeline 
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf, labelConverter])

# Learning the model using the training data
model = pipeline.fit(train)

tt = time() - t0
print("Réalisé en {} secondes".format(round(tt,3)))

Calculate the accuracy of the trained model

In [12]:
#Predictions calulation 
predictions = model.transform(test)

# Show a sample of the data 
predictions.sample(False, 0.0001 , seed = 222).toPandas()

,label,features,indexedLabel,indexedFeatures,rawPrediction,probability,prediction,predictedLabel
0,0,"[49427.0, 1.26814221540323, 0.284467105331616,...",0.0,"[49427.0, 1.26814221540323, 0.284467105331616,...","[19.990837994183423, 0.009162005816572866]","[0.9995418997091713, 0.00045810029082864336]",0.0,0
1,0,"[51026.0, -2.4135879993509, -3.40193598948058,...",0.0,"[51026.0, -2.4135879993509, -3.40193598948058,...","[19.99384694418137, 0.006153055818624167]","[0.9996923472090687, 0.0003076527909312084]",0.0,0
2,0,"[51159.0, 0.978630725214966, 0.34870441867448,...",0.0,"[51159.0, 0.978630725214966, 0.34870441867448,...","[19.99218964141947, 0.00781035858052759]","[0.9996094820709736, 0.0003905179290263796]",0.0,0
3,0,"[71262.0, -1.35577653242637, 0.740036151815906...",0.0,"[71262.0, -1.35577653242637, 0.740036151815906...","[19.995112415097125, 0.004887584902872326]","[0.9997556207548565, 0.0002443792451436163]",0.0,0
4,0,"[77103.0, -1.59534020576173, 0.073093776991043...",0.0,"[77103.0, -1.59534020576173, 0.073093776991043...","[19.98953838769776, 0.010461612302234717]","[0.9994769193848884, 0.0005230806151117361]",0.0,0
5,0,"[163555.0, 2.05320198269621, -0.04726665512059...",0.0,"[163555.0, 2.05320198269621, -0.04726665512059...","[19.995112415097125, 0.004887584902872326]","[0.9997556207548565, 0.0002443792451436163]",0.0,0
6,0,"[164587.0, -1.12593622931724, 0.71298685484290...",0.0,"[164587.0, -1.12593622931724, 0.71298685484290...","[19.99437900257373, 0.0056209974262653025]","[0.9997189501286867, 0.0002810498713132652]",0.0,0


In [ ]:
# Import of a MulticlassClassificationEvaluator from the package pyspark.ml.evaluation
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Creating an evaluator
evaluator = MulticlassClassificationEvaluator(metricName='accuracy',
                                              labelCol= 'indexedLabel',
                                              predictionCol= 'prediction')
# Calculation and display of the model accuracy 
accuracy = evaluator.evaluate(predictions)
print(accuracy)